In [1]:
import numpy as np
import pandas as pd
import sympy as sym
import math
from sympy.printing.theanocode import theano_function
import matplotlib.pyplot as plt
import utils
import _MEWtools as mt
import cycloidData

In [2]:
interior = utils.import_interior('interior1')

TOLERANCE = 1

cycloids = cycloidData.loadAllCycloids()
highResCycloids = cycloidData.loadAllCycloids(pointsPerCurve=10000)

AttributeError: module 'cycloidData' has no attribute 'loadAllCycloids'

In [4]:
import utils

overrides = {
    'obliquity': 0,
    'obliquityPhase': 0,
    'obliquityPhaseRate': 0,
    'spinRate': 0,
    'nonSynchronusRotationRate': 0,
    'librationAmplitude': 0,
    'librationPhase': 0,
    'librationFrequency': 0
}

sample_sat = utils.import_structure('Sample', overrides)

In [5]:
r, θ, φ, t = sym.symbols('r θ φ t', real = True)
stress = theano_function([t, φ, θ], (sample_sat.PC1, sample_sat.PC2, sample_sat.PCΨ, sample_sat.PCΨ2), 
                    dims={t: 1, φ: 1, θ:1}, 
                    dtypes={t: 'float64', φ: 'float64', θ:'float64'})

fnp = sym.lambdify([t, φ, θ], sample_sat.PC1, 'numpy')

In [12]:
stress2 = theano_function([t, φ, θ], (sample_sat.PC1, sample_sat.PC2, sample_sat.PCΨ2), 
                    dims={t: 1, φ: 1, θ:1}, 
                    dtypes={t: 'float64', φ: 'float64', θ:'float64'})

In [45]:
SIZE = 2000000

europa_orbit_seconds = 85 * 3600

time = np.random.default_rng().uniform(0, europa_orbit_seconds, SIZE)
colat = np.random.default_rng().uniform(0, np.pi, SIZE)
lon = np.random.default_rng().uniform(0, 2*np.pi, SIZE)

In [14]:
%timeit stress(time, colat, lon)
%timeit stress2(time, colat, lon)


5 s ± 78.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.23 s ± 163 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
results = stress(time, colat, lon)

In [63]:
results[0].shape
np.column_stack(results)

SyntaxError: invalid syntax (<ipython-input-63-ff0d995a57e2>, line 2)

In [ ]:
results = stress(time, colat, lon)
maxStress = np.maximum(results[0], results[1])
key = maxStress == results[0]
theta = np.select([key, np.invert(key)], [results[2], results[3]])

output = np.column_stack((maxStress, theta, colat, lon, time))
output

In [ ]:
df = pd.DataFrame(output, columns=['stress', 'heading', 'lat', 'lon', 'time'])

In [ ]:
def getMewStressField(times, colats, lons):
    results = stress(times, colats, lons)
    maxStress = np.maximum(results[0], results[1])
    key = maxStress == results[0]
    theta = np.select([key, np.invert(key)], [results[2], results[3]])

    return np.column_stack((maxStress, theta, colats, lons, times))

In [ ]:
def getMewFieldForCurve(curve, timeSteps):
    EUROPA_ORBIT_SECONDS = 85 * 3600
    
    points = curve[['lon', 'lat']].copy()
    points['lon'] = np.radians(360-points['lon'])
    points['lat'] = np.radians(90-points['lat'])
    
    times = np.arange(0, EUROPA_ORBIT_SECONDS, EUROPA_ORBIT_SECONDS/timeSteps)
    
    coord_index = np.array(points.index)
    combos = np.array(np.meshgrid(coord_index, times)).T.reshape(-1, 2)
    
    lons = np.array(points.iloc[combos[:, 0:1].flatten()]['lon'])
    lats = np.array(points.iloc[combos[:, 0:1].flatten()]['lat'])
    fullTimes = combos[:, 1:2].flatten()
    
    output = getMewStressField(fullTimes, lats, lons)
    df = pd.DataFrame(output, columns=['stress', 'heading', 'lat', 'lon', 'time'])
    df['lat'] = np.round(np.degrees(df['lat']), 4)
    df['lon'] = np.round(np.degrees(df['lon']), 4)
    
    return df

In [ ]:
curve = highResCycloids['delphi'].curve.copy()
curve['lon'] = np.radians(360-curve['lon'])
curve['lat'] = np.radians(90-curve['lat'])

coord_index = np.array(curve.index)
combos = np.array(np.meshgrid(coord_index, times)).T.reshape(-1, 2)


In [ ]:
lons = np.array(curve.iloc[combos[:, 0:1].flatten()]['lon'])
lats = np.array(curve.iloc[combos[:, 0:1].flatten()]['lat'])
fullTimes = combos[:, 1:2].flatten()

output = getMewStressField(fullTimes, lats, lons)
df = pd.DataFrame(output, columns=['stress', 'heading', 'lat', 'lon', 'time'])
df['lat'] = np.round(np.degrees(df['lat']), 4)
df['lon'] = np.round(np.degrees(df['lon']), 4)

In [ ]:
filtered = df.loc[(df['lat'] == 158.2) & (df['lon'] == 130.07)]
# filtered = df.loc[(df['time'] == 100)]
plt.plot(filtered['time'], filtered['stress'])

In [ ]:
%timeit output = getMewStressField(fullTimes, lats, lons)

In [ ]:
coord_index = np.array(curve.index)
combos = np.array(np.meshgrid(coord_index, times)).T.reshape(-1, 2)

np.array(curve.iloc[combos[:, 0:1].flatten()][['lon', 'lat']])
# combos[:, 0:1]

In [ ]:
highResCycloids['sidon'].curve.shape

In [ ]:
getMewFieldForCurve(highResCycloids['delphi'].curve, 360)

# MEW Tools

In [29]:
#from __future__ import division
import numpy as np
import sympy as sym
import math
#import time


r, θ, φ, t = sym.symbols(  'r θ φ t'    ,   real=True  )

G = 6.67e-11

# Fundamental Matrix for Elastic Solutions
def GetEmatFunction():
    L_, r_, g_, rho_, mu_ = sym.symbols('L_ r_ g_ rho_ mu_')
    matrix = sym.Matrix([
                        [                                             r_**(1+L_) ,                                r_**(-1+L_) , 0                      ,                                               r_**(-L_)   ,                                r_**(-2-L_) , 0                  ],
                        [ ((3+L_)/(L_*(1+L_)))                      * r_**(1+L_) , (1/L_)                       * r_**(-1+L_) , 0                      , -((-2+L_)/(L_*(1+L_)))                      * r_**(-L_)   , -(1/(1+L_))                  * r_**(-2-L_) , 0                  ],
                        [ (2*mu_*((-3+L_*(-1+L_))/L_) + r_*rho_*g_) * r_**(L_)   , (2*mu_*(-1+L_) + r_*rho_*g_) * r_**(-2+L_) , rho_     * r_**(L_)    , (2*mu_*((1-L_*(3+L_))/(1+L_)) + r_*rho_*g_) * r_**(-1-L_) , (2*mu_*(-2-L_) + r_*rho_*g_) * r_**(-3-L_) , rho_ * r_**(-1-L_) ],
                        [ 2*mu_*((2+L_)/(1+L_))                     * r_**(L_)   , 2*mu_*((-1+L_)/(L_))         * r_**(-2+L_) , 0                      , 2*mu_*((-1+L_)/(L_))                        * r_**(-1-L_) , 2*mu_*((2+L_)/(1+L_))        * r_**(-3-L_) , 0                  ],
                        [ 0                                                      , 0                                          ,            r_**(L_)    , 0                                                         , 0                                          ,        r_**(-1-L_) ],
                        [ 4*np.pi*G*rho_                           * r_**(1+L_) , 4*np.pi*G*rho_              * r_**(-1+L_) , (1+2*L_) * r_**(-1+L_) , 4*np.pi*G*rho_                             * r_**(-L_)   , 4*np.pi*G*rho_              * r_**(-2-L_) , 0                  ]
                        ])
    return sym.lambdify([L_, r_, g_, rho_, mu_], matrix)

Emat = GetEmatFunction()


# Fundamental Matrix for Elastic Solutions Invserse
def GetEmatINVFunction():
    L_, r_, g_, rho_, mu_ = sym.symbols('L_ r_ g_ rho_ mu_')
    matrix = sym.Matrix([
                        [ -(((2*mu_*(2+L_) - r_*rho_*g_)*L_*(1+L_))/(2*mu_*(3+4*L_*(2+L_))))         * r_**(-1-L_) , ((L_**2 *(1+L_)*(2+L_)) / (3+4*L_*(2+L_)))  * r_**(-1-L_) , -((L_*(1+L_))/(2*mu_*(3+4*L_*(2+L_)))) * r_**(-L_)  , ((L_**2 *(1+L_))/(2*mu_*(3+4*L_*(2+L_))))   * r_**(-L_)  , ((L_*(1+L_)*rho_)/(2*mu_*(3+4*L_*(2+L_))))  * r_**(-L_)  , 0                          ],
                        [  ((L_*(2*mu_*(L_*(3+L_)-1)-(1+L_)*r_*rho_*g_))/(2*mu_*(-1+4*L_**2)))       * r_**(1-L_)  , -((L_*(-1+L_)*(1+L_)**2)/(-1+4*L_**2))       * r_**(1-L_)  ,  ((L_*(1+L_))/(2*mu_*(-1+4*L_**2)))    * r_**(2-L_) , -(((-2+L_)*L_*(1+L_))/(2*mu_*(-1+4*L_**2))) * r_**(2-L_) , -((L_*(1+L_)*rho_)/(2*mu_*(-1+4*L_**2)))    * r_**(2-L_) , 0                          ],
                        [ -((4*np.pi*G*rho_)/(1+2*L_))                                              * r_**(1-L_)  , 0                                                         , 0                                                   , 0                                                        , 0                                                        , (1/(1+2*L_)) * r_**(1-L_)  ],
                        [  ((L_*(1+L_)*(2*mu_*(-1+L_) + r_*rho_*g_))/(2*mu_*(-1+4*L_**2)))           * r_**(L_)    , ((L_*(-1+L_)*(1+L_)**2)/(-1+4*L_**2))        * r_**(L_)    , -((L_*(1+L_))/(2*mu_*(-1+4*L_**2)))    * r_**(1+L_) , -((L_*(1+L_)**2)/(2*mu_*(-1+4*L_**2)))      * r_**(1+L_) , ((L_*(1+L_)*rho_)/(2*mu_*(-1+4*L_**2)))     * r_**(1+L_) , 0                          ],
                        [ -(((1+L_)*(2*mu_*(-3+L_*(-1+L_))+ L_*r_*rho_*g_))/(2*mu_*(3+4*L_*(2+L_)))) * r_**(2+L_)  , -((L_**2 *(1+L_)*(2+L_))/((3+4*L_*(2+L_)))) * r_**(2+L_)  , ((L_*(1+L_))/(2*mu_*(3+4*L_*(2+L_))))  * r_**(3+L_) , ((L_*(1+L_)*(3+L_))/(2*mu_*(3+4*L_*(2+L_)))) * r_**(3+L_) , -((L_*(1+L_)*rho_)/(2*mu_*(3+4*L_*(2+L_)))) * r_**(3+L_) , 0                          ],
                        [  ((4*np.pi*G*rho_)/(1+2*L_))                                              * r_**(2+L_)  , 0                                                         , 0                                                   , 0                                                        ,                                               r_**(1+L_) , -(1/(1+2*L_)) * r_**(2+L_) ]
                        ])
    return sym.lambdify([L_, r_, g_, rho_, mu_], matrix)

EmatINV = GetEmatINVFunction()

# Fundamental Matrix for Fluid Solutions
def GetFmatFunction():
    L_, r_, g_, rho_ = sym.symbols('L_ r_ g_ rho_')
    matrix = sym.Matrix([
                        [ -(1/g_)                                              * r_**L_      , -(1/g_)                                               * r_**(-1-L_) ],
                        [ ((4*np.pi*G*rho_*r_ - (4+L_)*g_)/(L_*(1+L_)*g_**2)) * r_**L_      , ((4*np.pi*G*rho_*r_ + (-3+L_)*g_)/(L_*(1+L_)*g_**2)) * r_**(-1-L_) ],
                        [ 0                                                                  , 0                                                                   ],
                        [ 0                                                                  , 0                                                                   ],
                        [                                                        r_**L_      ,                                                         r_**(-1-L_) ],
                        [ ((-4*np.pi*G*rho_*r_ + (1+2*L_)*g_)/(g_))           * r_**(-1+L_) , -((4*np.pi*G*rho_)/(g_))                             * r_**(-1-L_) ]
                        ])
    return sym.lambdify([L_, r_, g_, rho_], matrix)

Fmat = GetFmatFunction()

# Fundamental Matrix for Fluid Solutions Inverse
def GetFmatINVFunction():
    L_, r_, g_, rho_ = sym.symbols('L_ r_ g_ rho_')
    matrix = sym.Matrix([
                        [ ((4*np.pi*G*rho_)/((1+2*L_)*g_))                  * r_**(1-L_) ,  (1/(1+2*L_)) * r_**(1-L_) ],
                        [ (((1+2*L_)*g_ - 4*np.pi*G*rho_*r_)/((1+2*L_)*g_)) * r_**(1+L_) , -(1/(1+2*L_)) * r_**(2+L_) ]
                        ])
    return sym.lambdify([L_, r_, g_, rho_], matrix)

FmatINV = GetFmatINVFunction()

############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
#                                 Love Number / y-Function Calculator                                      #
def lovefunc(L_, rin_, pin_, uin_, nin_, w_):                                                              #

    # Number of layers in the body
    layno = len(rin_)


    # Find Purely fluid or elastic layers and create the index lists
    fluidlayerlist   = np.where(np.array(uin_) == 0)[0]
    elasticlayerlist = np.where(np.array(nin_) == 0)[0]


    # If there are no fluid layers, then create this list instead
    if len(fluidlayerlist) == 0:
        fluidlayerlist = [None]


    # Turn rigidity into viscoelastic parameter for those layers
    uV=[]
    for i in range(layno):
        if i in elasticlayerlist:
            uV.append(uin_[i])
        else:
            uV.append(uin_[i] / (1-1j*(uin_[i]/(nin_[i]*w_)) ))


    # Make a new lists of inputs
    rlist=rin_
    plist=pin_
    ulist=uV


    # Gravitational Acceleration Function
    def gfunc(r_, lay_):

        rsw0 = np.insert(rlist,0,0)

        gcalc = plist[lay_] * (r_**3 - rsw0[lay_]**3)

        for i in range(0,lay_):
            gcalc += plist[i] * (rsw0[i+1]**3 - rsw0[i]**3)

        return ((4*np.pi*G)/(3*r_**2)) * gcalc


    # Make empty lists for boundary conditions resulting from elastic->fluid interface
    fluidBC     = []
    fluidBCvals = []


    # Make empty lists for each layer's fundimental matrix
    mats=[None]*(layno)


    # Determine if the core is fluid or not & produce core matrix
    if fluidlayerlist[0]==0:
        mats[0] = Fmat(L_, rlist[0], gfunc(rlist[0], 0), plist[0])[:,0]
    else:
        mats[0] = Emat(L_, rlist[0], gfunc(rlist[0], 0), plist[0], ulist[0])[:,0:3]


    # Produce the matrix for all the other layers
    for i in range(1,layno):
        if i-1 in fluidlayerlist and i in fluidlayerlist:

            mats[i] = (Fmat(L_, rlist[i], gfunc(rlist[i], i), plist[i]) @ FmatINV(L_, rlist[i-1], gfunc(rlist[i-1], i-1), plist[i]) @ mats[i-1][4:,:])
        #            print "fluid fluid"


        elif i-1 in fluidlayerlist and i not in fluidlayerlist:

            mattemp = mats[i-1][:,:]
            mattemp[1:2,:]=sym.zeros(1,len(mattemp[1:2,:]))
            rhs = np.array([[-1,0],[0,1],[-plist[i-1]*gfunc(rlist[i-1],i-1),0],[0,0],[0,0],[-4 *np.pi *G *plist[i-1],0]])
            mattemp2 = np.hstack((mattemp, rhs)) #sym.Matrix([[-1,0],[0,1],[-plist[i-1]*gfunc(rlist[i-1],i-1),0],[0,0],[0,0],[-4 *np.pi *G *plist[i-1],0]]).col_insert(0,mattemp)

            mats[i] = (Emat(L_, rlist[i], gfunc(rlist[i], i), plist[i], ulist[i]) @ EmatINV(L_, rlist[i-1], gfunc(rlist[i-1], i-1), plist[i], ulist[i]) @ mattemp2)
        #            print "fluid elastic"


        elif i-1 not in fluidlayerlist and i in fluidlayerlist:

            fluidBC.append((mats[i-1][2:3,:] - plist[i] * gfunc(rlist[i-1], i-1) * ((mats[i-1][4:5,:] / gfunc(rlist[i-1], i-1)) + mats[i-1][0:1,:])))

            fluidBC.append(mats[i-1][3:4,:])

            fluidBCvals.extend([[0], [0]])

            mats[i] = (Fmat(L_, rlist[i], gfunc(rlist[i], i), plist[i]) @ FmatINV(L_, rlist[i-1], gfunc(rlist[i-1], i-1), plist[i]) \
                 @ np.insert((mats[i-1][5:6,:] - 4*np.pi*G*plist[i]*((mats[i-1][4:5,:] / gfunc(rlist[i-1], i-1)) + mats[i-1][0:1,:])), 0, mats[i-1][4:5,:], 0))

        #            print "elastic fluid"


        elif i-1 not in fluidlayerlist and i not in fluidlayerlist:

            mats[i] = (Emat(L_, rlist[i], gfunc(rlist[i], i), plist[i], ulist[i]) @ EmatINV(L_, rlist[i-1], gfunc(rlist[i-1], i-1), plist[i], ulist[i]) @ mats[i-1])
    #            print "elastic elastic"



    # Give that matrix right at the body surface
    surfmat = mats[-1]
    #    print "surface matrix"

    # Pad the list of elastic->fluid interface boundary condition with zeros to the right (so all rows have the same ammt of columns for BC constants)
    for i in range(len(fluidBC)):
        fluidBC[i] = np.lib.pad(fluidBC[i],((0,0),(0,surfmat[2:3,:].shape[1]-fluidBC[i].shape[1])),'constant')

    # Recast the e->f BC list as a matrix
    fluidBC = np.concatenate(fluidBC)

    # Join the surface BCs to the e->f BCs
    if fluidlayerlist[-1]==layno-1:
        BCmat  = np.concatenate([fluidBC, surfmat[5:6,:]])
        BCvals = np.concatenate([fluidBCvals, [[-(2*L_+1)/rlist[-1]]]])
    elif fluidBC == []:
        BCmat  = np.concatenate([surfmat[2:3,:],surfmat[3:4,:],surfmat[5:6,:]])
        BCvals = np.concatenate([[0],[0],[-(2*L_+1)/rlist[-1]]])
    else:
        BCmat = np.concatenate([fluidBC, surfmat[2:3,:],surfmat[3:4,:],surfmat[5:6,:]])
        BCvals = np.concatenate([fluidBCvals, [[0],[0],[-(2*L_+1)/rlist[-1]]]])
    #    print "BC stuff"


    # Solve for the constants of integration
    # consts = (BCmat.inv() * BCvals).applyfunc(sym.simplify)
    consts = np.linalg.solve(BCmat, BCvals)

    #    print "constants"




    # Calculate the Love numbers
    lovenums = np.array([gfunc(rlist[-1], layno-1)*(surfmat[0:1,:]@consts)[0],
                gfunc(rlist[-1], layno-1)*(surfmat[1:2,:]@consts)[0],
                -((surfmat[4:5,:]@consts)[0] + 1)]).flatten()
    lovenums = sym.Array(lovenums)
#    print "love numbers"



    # Calculate the y-functions in every layer
    # yfuncs = []
    # for i in range(len(mats)):
    #     yfuncs.append(sum((mats[i]*sym.Matrix(consts[:len(mats[i][0:1,:])])).applyfunc(sym.expand).tolist(),[]))
#    print "y functions"


    return lovenums


############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
#                                 Tide Calculator

#n, S, N, Ln, opn, t = sym.symbols('n S N Ln opn t',real=True)

def freqsinorder(n_, S_, N_, Ln_, opn_):
    return np.abs([ 2*n_ - 2*S_ - 2*N_ + 0*Ln_ + 0*opn_,
             1*n_ - 2*S_ - 2*N_ + 0*Ln_ + 0*opn_,
             3*n_ - 2*S_ - 2*N_ + 0*Ln_ + 0*opn_,
             0*n_ + 2*S_ + 2*N_ + 0*Ln_ - 2*opn_,
             1*n_ - 2*S_ - 2*N_ + 0*Ln_ + 2*opn_,
             1*n_ + 2*S_ + 2*N_ + 0*Ln_ - 2*opn_,
             2*n_ + 2*S_ + 2*N_ + 0*Ln_ - 4*opn_,
             3*n_ + 2*S_ + 2*N_ + 0*Ln_ - 4*opn_,
             1*n_ + 2*S_ + 2*N_ + 0*Ln_ - 4*opn_,
            -2*n_ + 2*S_ + 2*N_ + 1*Ln_ - 0*opn_,
             2*n_ - 2*S_ - 2*N_ + 1*Ln_ - 0*opn_,
            -2*n_ - 2*S_ - 2*N_ + 1*Ln_ + 4*opn_,
             0*n_ + 2*S_ + 2*N_ + 1*Ln_ - 2*opn_,
             0*n_ - 2*S_ - 2*N_ + 1*Ln_ + 2*opn_,
             2*n_ + 2*S_ + 2*N_ + 1*Ln_ - 4*opn_,
             2*n_ - 1*S_ - 1*N_ + 0*Ln_ - 1*opn_,
             3*n_ - 1*S_ - 1*N_ + 0*Ln_ - 1*opn_,
             1*n_ - 1*S_ - 1*N_ + 0*Ln_ - 1*opn_,
             0*n_ + 1*S_ + 1*N_ + 0*Ln_ - 1*opn_,
             1*n_ + 1*S_ + 1*N_ + 0*Ln_ - 1*opn_,
             1*n_ - 1*S_ - 1*N_ + 0*Ln_ + 1*opn_,
             2*n_ + 1*S_ + 1*N_ + 0*Ln_ - 3*opn_,
             3*n_ + 1*S_ + 1*N_ + 0*Ln_ - 3*opn_,
             1*n_ + 1*S_ + 1*N_ + 0*Ln_ - 3*opn_,
             2*n_ + 1*S_ + 1*N_ + 1*Ln_ - 3*opn_,
             0*n_ + 1*S_ + 1*N_ + 1*Ln_ - 1*opn_,
             2*n_ - 1*S_ - 1*N_ + 1*Ln_ - 1*opn_,
            -2*n_ + 1*S_ + 1*N_ + 1*Ln_ + 1*opn_,
             0*n_ - 1*S_ - 1*N_ + 1*Ln_ + 1*opn_,
            -2*n_ - 1*S_ - 1*N_ + 1*Ln_ + 3*opn_,
             1*n_ + 0*S_ + 0*N_ + 0*Ln_ + 0*opn_,
             2*n_ + 0*S_ + 0*N_ + 0*Ln_ - 2*opn_,
             1*n_ + 0*S_ + 0*N_ + 0*Ln_ - 2*opn_,
             3*n_ + 0*S_ + 0*N_ + 0*Ln_ - 2*opn_,
             0*n_ + 0*S_ + 0*N_ + 0*Ln_ + 0*opn_])


def potsinorder(G_, M_, R_, a_, e_, o_, op_, LA_, Lp_, n_, S_, N_, Ln_, opn_, t_ ,theta_, phi_):
    return [  ((3*G_*M_*R_**2)/(4*a_**3))   *              sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[0]*t_ - 2*phi_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[1]*t_ - 2*phi_),
             ((21*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[2]*t_ - 2*phi_),
              ((3*G_*M_*R_**2)/(8*a_**3))   *              sym.sin(o_)**2     *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[3]*t_ + 2*phi_ - 2*op_),
              ((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(o_)**2     *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[4]*t_ - 2*phi_ + 2*op_),
              ((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(o_)**2     *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[5]*t_ + 2*phi_ - 2*op_),
              ((3*G_*M_*R_**2)/(4*a_**3))   *              sym.sin(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[6]*t_ + 2*phi_ - 4*op_),
             ((21*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.sin(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[7]*t_ + 2*phi_ - 4*op_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.sin(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[8]*t_ + 2*phi_ - 4*op_),
              ((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[9]*t_ + 2*phi_ + 0*op_ + Lp_),
             -((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.cos(o_/2)**4                *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[10]*t_ - 2*phi_ + 0*op_ + Lp_),
             -((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.sin(o_/2)**4                *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[11]*t_ - 2*phi_ + 4*op_ + Lp_),
              ((3*G_*M_*R_**2)/(8*a_**3))   * e_ * LA_ *   sym.sin(o_)**2                  *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[12]*t_ + 2*phi_ - 2*op_ + Lp_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ * LA_ *   sym.sin(o_)**2                  *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[13]*t_ - 2*phi_ + 2*op_ + Lp_),
              ((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.sin(o_/2)**4                *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[14]*t_ + 2*phi_ - 4*op_ + Lp_),
             -((3*G_*M_*R_**2)/(16*a_**3))  *              (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[15]*t_ - 1*phi_ - 1*op_),
             -((21*G_*M_*R_**2)/(32*a_**3)) * e_ *         (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[16]*t_ - 1*phi_ - 1*op_),
              ((3*G_*M_*R_**2)/(32*a_**3))  * e_ *         (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[17]*t_ - 1*phi_ - 1*op_),
             -((3*G_*M_*R_**2)/(8*a_**3))   *              sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[18]*t_ + 1*phi_ - 1*op_),
             -((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[19]*t_ + 1*phi_ - 1*op_),
              ((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[20]*t_ - 1*phi_ + 1*op_),
             -((3*G_*M_*R_**2)/(2*a_**3))   *              sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[21]*t_ + 1*phi_ - 3*op_),
             -((21*G_*M_*R_**2)/(4*a_**3))  * e_ *         sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[22]*t_ + 1*phi_ - 3*op_),
             ((3*G_*M_*R_**2)/(4*a_**3))    * e_ *         sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[23]*t_ + 1*phi_ - 3*op_),
             -((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[24]*t_ + 1*phi_ - 3*op_ + Lp_),
             -((3*G_*M_*R_**2)/(16*a_**3))   * e_ * LA_ *  sym.sin(2*o_)                  *   sym.sin(2*theta_)                       *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[25]*t_ + 1*phi_ - 1*op_ + Lp_),
              ((3*G_*M_*R_**2)/(32*a_**3))   * e_ * LA_ *  (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[26]*t_ - 1*phi_ - 1*op_ + Lp_),
              ((3*G_*M_*R_**2)/(32*a_**3))   * e_ * LA_ *  (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[27]*t_ + 1*phi_ + 1*op_ + Lp_),
             -((3*G_*M_*R_**2)/(16*a_**3))   * e_ * LA_ *  sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[28]*t_ - 1*phi_ + 1*op_ + Lp_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ * LA_ *  sym.sin(o_) * sym.sin(o_/2)**2   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[29]*t_ - 1*phi_ + 3*op_ + Lp_),
             -((3*G_*M_*R_**2)/(32*a_**3))   * e_ *       (1 + 3*sym.cos(2*o_))            *   (1 + 3*sym.cos(2*theta_))              *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[30]*t_ ),
              ((3*G_*M_*R_**2)/(8*a_**3))   *             sym.sin(o_)**2                   *   (1 - 3*sym.cos(theta_)**2)             *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[31]*t_ - 2*op_),
              ((3*G_*M_*R_**2)/(32*a_**3))  * e_ *        sym.sin(o_)**2                   *   (1 + 3*sym.cos(2*theta_))              *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[32]*t_ - 2*op_),
             ((21*G_*M_*R_**2)/(16*a_**3))  * e_ *        sym.sin(o_)**2                   *   (1 - 3*sym.cos(theta_)**2)             *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[33]*t_ - 2*op_),
             -((G_*M_*R_**2)/(32*a_**3))    *             (1 + 3*sym.cos(2*o_))            *   (1 + 3*sym.cos(2*theta_))              *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[34]*t_ )]


############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
#                                 Create the Satellite Class                                               #
class satellite:                                                                                           #

    def __init__(self,rin=[],pin=[],uin=[],nin=[],G=0,M=0,a=0,e=0,o=0,op=0,on=0,Ω=0,NSRn=0,libA=0,libp=0,libn=0):


        #All The Inputs
        self.rin = rin
        self.pin = pin
        self.uin = uin
        self.nin = nin

        self.G = G
        self.M = M
        self.a = a

        self.e = e



        self.o    = o
        self.op   = op
        self.on   = on


        self.NSRn    = NSRn


        self.libA   = libA
        self.libp   = libp
        self.libn   = libn


        # Body Radius
        self.R = self.rin[-1]

        # Body Mass
        self.mass = (4*np.pi/3) * pin[0]*rin[0]**3
        for i in range(1,len(rin)):
            self.mass += (4*np.pi/3) * pin[i]*(rin[i]**3 - rin[i-1]**3)

        # Mean Motion
        self.n = np.sqrt(self.G *(self.mass + self.M) / self.a**3)

        # grav accel
        self.g = self.G*self.mass / self.R**2


        if Ω == 0:
            self.Ω = self.n
        else:
            self.Ω = Ω



        self.freqlist = freqsinorder(self.n, self.Ω, self.NSRn, self.libn, self.on)
        self.potslist = potsinorder(self.G, self.M, self.R, self.a, self.e, self.o, self.op, self.libA, self.libp, self.n, self.Ω, self.NSRn, self.libn, self.on, t ,θ, φ)


        self.freqlist2 = []
        self.potslist2 = []
        self.static = []
        for i in range(len(self.freqlist)):
            if self.potslist[i] == 0:
                continue
            else:
                if self.freqlist[i] == 0:
                    self.static.append(self.potslist[i])
                else:
                    self.potslist2.append(self.potslist[i])
                    self.freqlist2.append(self.freqlist[i])

        self.static = [sum(self.static)]


        self.lovelist = []
        for i in self.freqlist2:
            self.lovelist.append(lovefunc(2, self.rin, self.pin, self.uin, self.nin, i))




        self.tt = 0
        self.pp = 0
        self.tp = 0
        for i in range(len(self.freqlist2)):

            #Surface Rigidity
            if self.nin[-1] == 0:
                self.uR = self.uin[-1]
            else:
                self.uR =  self.uin[-1] / (1-1j*(self.uin[-1]/(self.nin[-1]*self.freqlist2[i])))

            POT = self.potslist2[i]
            h = self.lovelist[i][0]
            l = self.lovelist[i][1]

            self.tt += ((2.0*self.uR)/(self.R*self.g)) * ((3.0*h -  6.0*l)*POT + l*sym.diff(POT,θ,θ))
            self.pp += ((2.0*self.uR)/(self.R*self.g)) * ((3.0*h - 12.0*l)*POT - l*sym.diff(POT,θ,θ))
            self.tp += ((2.0*self.uR)/(self.R*self.g)) *l * sym.csc(θ)*(sym.diff(POT,θ,φ) - sym.atan(θ)*sym.diff(POT,φ))

        self.ttR = sym.re(self.tt)
        self.ppR = sym.re(self.pp)
        self.tpR = sym.re(self.tp)

        self.PC1  = (1/2) * (self.ttR + self.ppR + sym.sqrt(4*self.tpR**2 + (self.ttR-self.ppR)**2))
        self.PC2  = (1/2) * (self.ttR + self.ppR - sym.sqrt(4*self.tpR**2 + (self.ttR-self.ppR)**2))
        self.PCΨ  = (1/2) * sym.atan( (2*self.tpR)/(self.ttR-self.ppR))
        self.PCΨ2 = (1/2) * sym.atan2((2*self.tpR),(self.ttR-self.ppR))




In [30]:
import yaml
import pathlib
import os

STRUCTURE_PATH = 'structures'
INTERIOR_PATH = 'interiors'

def import_structure(name, overrides={}):
    folder = './'
    path = os.path.join(folder, STRUCTURE_PATH, f'{name}.yml')
    
    with open(path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except:
            config = {}

    config.update(overrides)

    radii = list(map(float, config['radii']))
    density = list(map(float, config['density']))
    rigidity = list(map(float, config['rigidity']))
    viscosity = list(map(float, config['viscosity']))

    G = 6.67e-11

    mass = float(config['mass']) if config.get('mass') is not None else 0
    semiMajorDistance = float(config['semiMajorDistance']) if config.get('semiMajorDistance') is not None else 0
    eccentricity = float(config['eccentricity']) if config.get('eccentricity') is not None else 0
    obliquity = float(config['obliquity']) if config.get('obliquity') is not None else 0
    obliquityPhase = float(config['obliquityPhase']) if config.get('obliquityPhase') is not None else 0
    obliquityPhaseRate = float(config['obliquityPhaseRate']) if config.get('obliquityPhaseRate') is not None else 0

    spinRate = float(config['spinRate']) if config.get('spinRate') is not None else 0

    nonSynchronusRotationRate = float(config['nonSynchronusRotationRate']) if config.get('nonSynchronusRotationRate') is not None else 0
    librationAmplitude = float(config['librationAmplitude']) if config.get('librationAmplitude') is not None else 0
    librationPhase = float(config['librationPhase']) if config.get('librationPhase') is not None else 0
    librationFrequency = float(config['librationFrequency']) if config.get('librationFrequency') is not None else 0

    sat = satellite(
        rin = radii,
        pin = density,
        uin = rigidity,
        nin = viscosity,
        G = G,
        M = mass,
        a = semiMajorDistance,
        e = eccentricity,
        o = obliquity,
        op = obliquityPhase,
        on = obliquityPhaseRate,
        Ω = spinRate,
        NSRn = nonSynchronusRotationRate,
        libA = librationAmplitude,
        libp = librationPhase,
        libn = librationFrequency
    )
    return sat

In [31]:
import numpy as np
import pandas as pd
import sympy as sym
import math
from sympy.printing.theanocode import theano_function
import matplotlib.pyplot as plt
import utils
import _MEWtools as mt
import cProfile
import pstats

def test_perf():
    import_structure('Sample')
    
cProfile.run('test_perf()', 'perf.log')

p = pstats.Stats('perf.log')
p.sort_stats('cumulative').print_stats(20)

Tue May 26 21:57:21 2020    perf.log

         96172 function calls (91712 primitive calls) in 0.080 seconds

   Ordered by: cumulative time
   List reduced from 548 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.080    0.080 {built-in method builtins.exec}
        1    0.000    0.000    0.080    0.080 <string>:1(<module>)
        1    0.000    0.000    0.080    0.080 <ipython-input-31-4a9aef62d789>:12(test_perf)
        1    0.000    0.000    0.080    0.080 <ipython-input-30-bd28f9dd2eca>:8(import_structure)
        1    0.000    0.000    0.073    0.073 <ipython-input-29-62768ed070a7>:322(__init__)
       12    0.000    0.000    0.050    0.004 /Users/stan/Applications/Anaconda/anaconda3/envs/europa_cycloids/lib/python3.7/site-packages/sympy/core/function.py:2384(diff)
       12    0.000    0.000    0.050    0.004 /Users/stan/Applications/Anaconda/anaconda3/envs/europa_cycloids/lib/python3.7/site-p

/Users/stan/Applications/Anaconda/anaconda3/envs/europa_cycloids/lib/python3.7/site-packages/ipykernel_launcher.py:191: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [34]:
%timeit test_perf()

/Users/stan/Applications/Anaconda/anaconda3/envs/europa_cycloids/lib/python3.7/site-packages/ipykernel_launcher.py:191: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


33.8 ms ± 986 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
L_, rin_, pin_, uin_, nin_, w_ = sym.symbols('L_ rin_ pin_ uin_ nin_ w_', real = True)

In [ ]:
val = sym.Matrix([1, 2, 3])
L_, rin_, pin_, uin_, nin_, w_ = sym.symbols(
    'L_ rin_ pin_ uin_ nin_ w_', real=True)

run = sym.MatrixSymbol('rin_')

len(rin_)

In [ ]:
mat = sym.Matrix([1, 2, 3.])
mat == sym.Matrix(mat)

In [1]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time
 
vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000
 
rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print('Looping %d times took' % iters, t1 - t0, 'seconds')
print('Result is', r)
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 1.30072021484375 seconds
Result is [1.23178032 1.61879341 1.52278065 ... 2.20771815 2.29967753 1.62323285]
Used the cpu


In [40]:
sat = import_structure('Sample')
f = sym.lambdify([t, φ, θ], sat.PC1)

f(1, 1, 1)

/Users/stan/Applications/Anaconda/anaconda3/envs/europa_cycloids/lib/python3.7/site-packages/ipykernel_launcher.py:191: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


24971.074534608222

In [42]:
SIZE = 100

europa_orbit_seconds = 85 * 3600

times = np.random.default_rng().uniform(0, europa_orbit_seconds, SIZE)
colat = np.random.default_rng().uniform(0, np.pi, SIZE)
lon = np.random.default_rng().uniform(0, 2*np.pi, SIZE)

df = pd.DataFrame(colat, columns = ['lat'])
df['lon'] = lon

# df2 = pd.DataFrame([(point.lat, point.lon, time) for point in df.itertuples() for time in times],
#                   columns = ['lat', 'lon', 'time'])
# df2.sort_values(['lat', 'lon', 'time'])

results = np.array([[np.degrees(point.lat), np.degrees(point.lon), time] for point in df.itertuples() for time in times])
results

array([[5.24861112e+01, 1.10138588e+02, 6.17084383e+04],
       [5.24861112e+01, 1.10138588e+02, 1.33731593e+05],
       [5.24861112e+01, 1.10138588e+02, 1.50693988e+04],
       ...,
       [4.31252484e+01, 2.84764708e+02, 5.99566991e+04],
       [4.31252484e+01, 2.84764708e+02, 4.65307902e+04],
       [4.31252484e+01, 2.84764708e+02, 1.18602391e+05]])

In [32]:
steps = np.linspace(1, europa_orbit_seconds, 360)
steps

array([1.00000000e+00, 8.53364903e+02, 1.70572981e+03, 2.55809471e+03,
       3.41045961e+03, 4.26282451e+03, 5.11518942e+03, 5.96755432e+03,
       6.81991922e+03, 7.67228412e+03, 8.52464903e+03, 9.37701393e+03,
       1.02293788e+04, 1.10817437e+04, 1.19341086e+04, 1.27864735e+04,
       1.36388384e+04, 1.44912033e+04, 1.53435682e+04, 1.61959331e+04,
       1.70482981e+04, 1.79006630e+04, 1.87530279e+04, 1.96053928e+04,
       2.04577577e+04, 2.13101226e+04, 2.21624875e+04, 2.30148524e+04,
       2.38672173e+04, 2.47195822e+04, 2.55719471e+04, 2.64243120e+04,
       2.72766769e+04, 2.81290418e+04, 2.89814067e+04, 2.98337716e+04,
       3.06861365e+04, 3.15385014e+04, 3.23908663e+04, 3.32432312e+04,
       3.40955961e+04, 3.49479610e+04, 3.58003259e+04, 3.66526908e+04,
       3.75050557e+04, 3.83574206e+04, 3.92097855e+04, 4.00621504e+04,
       4.09145153e+04, 4.17668802e+04, 4.26192451e+04, 4.34716100e+04,
       4.43239749e+04, 4.51763398e+04, 4.60287047e+04, 4.68810696e+04,
      

range(1, 360)